# SVM Theory and Mathematical Foundations

This notebook provides a comprehensive overview of Support Vector Machine (SVM) theory, mathematical foundations, and intuitive explanations.

## Table of Contents
1. [Introduction to SVMs](#introduction)
2. [Mathematical Foundations](#math-foundations)
3. [The Kernel Trick](#kernel-trick)
4. [Optimization Problem](#optimization)
5. [Visual Intuition](#visual-intuition)
6. [Practical Considerations](#practical)

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import sys
import os

# Add project root to path
sys.path.append(os.path.abspath('..'))

# Import our custom SVM implementations
from src.svm.linear_svm import LinearSVM
from src.svm.kernel_svm import KernelSVM
from src.svm.kernels import *
from src.utils.visualization import SVMVisualizer

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Jupyter notebook display settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 1. Introduction to SVMs {#introduction}

Support Vector Machines (SVMs) are powerful supervised learning algorithms used for both classification and regression tasks. The key idea behind SVMs is to find the optimal hyperplane that separates different classes with the maximum margin.

### Key Concepts:
- **Hyperplane**: A decision boundary that separates different classes
- **Support Vectors**: Data points closest to the hyperplane
- **Margin**: Distance between the hyperplane and the nearest data points
- **Maximum Margin**: The largest possible margin between classes

In [ ]:
# Create a simple 2D dataset to illustrate SVM concepts
np.random.seed(42)

# Generate linearly separable data
class1 = np.random.multivariate_normal([2, 2], [[0.5, 0], [0, 0.5]], 20)
class2 = np.random.multivariate_normal([6, 6], [[0.5, 0], [0, 0.5]], 20)

X_simple = np.vstack([class1, class2])
y_simple = np.hstack([np.ones(20), -np.ones(20)])

# Visualize the data
plt.figure(figsize=(10, 8))
plt.scatter(class1[:, 0], class1[:, 1], c='red', marker='o', s=100, 
           alpha=0.7, label='Class +1')
plt.scatter(class2[:, 0], class2[:, 1], c='blue', marker='s', s=100, 
           alpha=0.7, label='Class -1')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Simple Linearly Separable Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Mathematical Foundations {#math-foundations}

### 2.1 Linear SVM Formulation

For a binary classification problem, we want to find a hyperplane that separates the data:

$$f(x) = w^T x + b$$

where:
- $w$ is the weight vector (normal to the hyperplane)
- $b$ is the bias term
- The decision rule is: $\text{sign}(f(x))$

### 2.2 Margin Maximization

The margin for a point $(x_i, y_i)$ is:

$$\text{margin}_i = \frac{y_i(w^T x_i + b)}{||w||}$$

We want to maximize the minimum margin:

$$\max_{w,b} \min_i \frac{y_i(w^T x_i + b)}{||w||}$$

### 2.3 Primal Optimization Problem

This leads to the primal optimization problem:

$$\min_{w,b} \frac{1}{2}||w||^2$$
$$\text{subject to: } y_i(w^T x_i + b) \geq 1, \forall i$$

In [ ]:
# Demonstrate the concept of margin
def plot_svm_margin_concept():
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Create simple data
    X = np.array([[1, 2], [2, 3], [3, 3], [6, 5], [7, 6], [8, 7]])
    y = np.array([1, 1, 1, -1, -1, -1])
    
    # Different hyperplanes
    hyperplanes = [
        {'w': np.array([1, 0]), 'b': -4.5, 'title': 'Poor Separation'},
        {'w': np.array([1, 1]), 'b': -6, 'title': 'Better Separation'},
        {'w': np.array([0.5, 1]), 'b': -4, 'title': 'Maximum Margin (SVM)'}
    ]
    
    for idx, (ax, hp) in enumerate(zip(axes, hyperplanes)):
        # Plot data points
        pos_mask = y == 1
        neg_mask = y == -1
        
        ax.scatter(X[pos_mask, 0], X[pos_mask, 1], c='red', marker='o', 
                  s=100, alpha=0.7, label='Class +1')
        ax.scatter(X[neg_mask, 0], X[neg_mask, 1], c='blue', marker='s', 
                  s=100, alpha=0.7, label='Class -1')
        
        # Plot hyperplane
        x_range = np.linspace(0, 9, 100)
        if hp['w'][1] != 0:
            y_range = -(hp['w'][0] * x_range + hp['b']) / hp['w'][1]
            ax.plot(x_range, y_range, 'k-', linewidth=2, label='Decision Boundary')
            
            # Plot margin lines for the SVM case
            if idx == 2:
                y_margin_pos = -(hp['w'][0] * x_range + hp['b'] - 1) / hp['w'][1]
                y_margin_neg = -(hp['w'][0] * x_range + hp['b'] + 1) / hp['w'][1]
                ax.plot(x_range, y_margin_pos, 'k--', alpha=0.5, label='Margin')
                ax.plot(x_range, y_margin_neg, 'k--', alpha=0.5)
        
        ax.set_xlim(0, 9)
        ax.set_ylim(1, 8)
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.set_title(hp['title'])
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_svm_margin_concept()

### 2.4 Dual Formulation

Using Lagrange multipliers, we can derive the dual problem:

$$\max_{\alpha} \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \alpha_i \alpha_j y_i y_j x_i^T x_j$$

$$\text{subject to: } \sum_{i=1}^n \alpha_i y_i = 0, \quad \alpha_i \geq 0$$

The optimal weight vector is:
$$w^* = \sum_{i=1}^n \alpha_i^* y_i x_i$$

Points with $\alpha_i > 0$ are called **support vectors**.

In [ ]:
# Train a linear SVM and visualize support vectors
svm = LinearSVM(C=1.0, max_iter=1000)
svm.fit(X_simple, y_simple)

# Create visualization
visualizer = SVMVisualizer()
fig = visualizer.plot_decision_boundary_2d(svm, X_simple, y_simple, 
                                          title="Linear SVM with Support Vectors")
plt.show()

print(f"Number of support vectors: {len(svm.support_vectors_)}")
print(f"Support vector indices: {svm.support_vector_indices_}")

## 3. The Kernel Trick {#kernel-trick}

For non-linearly separable data, we can map the input space to a higher-dimensional feature space using a kernel function:

$$K(x_i, x_j) = \phi(x_i)^T \phi(x_j)$$

The dual formulation becomes:
$$\max_{\alpha} \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \alpha_i \alpha_j y_i y_j K(x_i, x_j)$$

### Common Kernel Functions:

1. **Linear**: $K(x_i, x_j) = x_i^T x_j$
2. **Polynomial**: $K(x_i, x_j) = (\gamma x_i^T x_j + r)^d$
3. **RBF (Gaussian)**: $K(x_i, x_j) = \exp(-\gamma ||x_i - x_j||^2)$
4. **Sigmoid**: $K(x_i, x_j) = \tanh(\gamma x_i^T x_j + r)$

In [ ]:
# Demonstrate kernel functions
def visualize_kernel_functions():
    # Create test points
    x = np.linspace(-3, 3, 100)
    x_ref = np.array([0])  # Reference point
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    kernels = [
        ('Linear', lambda xi: linear_kernel(xi.reshape(-1, 1), x_ref.reshape(-1, 1)).flatten()),
        ('Polynomial (d=3)', lambda xi: polynomial_kernel(xi.reshape(-1, 1), x_ref.reshape(-1, 1), degree=3, gamma=1, coef0=1).flatten()),
        ('RBF (γ=1)', lambda xi: rbf_kernel(xi.reshape(-1, 1), x_ref.reshape(-1, 1), gamma=1).flatten()),
        ('Sigmoid', lambda xi: sigmoid_kernel(xi.reshape(-1, 1), x_ref.reshape(-1, 1), gamma=1, coef0=0).flatten())
    ]
    
    for idx, (name, kernel_func) in enumerate(kernels):
        y = kernel_func(x)
        axes[idx].plot(x, y, linewidth=3)
        axes[idx].axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Reference point')
        axes[idx].set_title(f'{name} Kernel')
        axes[idx].set_xlabel('Input value')
        axes[idx].set_ylabel('Kernel value')
        axes[idx].grid(True, alpha=0.3)
        axes[idx].legend()
    
    plt.tight_layout()
    plt.show()

visualize_kernel_functions()

In [ ]:
# Create non-linearly separable data
np.random.seed(42)

# Generate circular data
n_samples = 200
theta = np.linspace(0, 2*np.pi, n_samples)
r1 = 2 + 0.5 * np.random.randn(n_samples//2)
r2 = 4 + 0.5 * np.random.randn(n_samples//2)

X_circle = np.vstack([
    np.column_stack([r1 * np.cos(theta[:n_samples//2]), r1 * np.sin(theta[:n_samples//2])]),
    np.column_stack([r2 * np.cos(theta[n_samples//2:]), r2 * np.sin(theta[n_samples//2:])])
])
y_circle = np.hstack([np.ones(n_samples//2), -np.ones(n_samples//2)])

# Compare linear vs RBF kernel
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Linear SVM
svm_linear = KernelSVM(kernel='linear', C=1.0)
svm_linear.fit(X_circle, y_circle)

visualizer.plot_decision_boundary_2d(svm_linear, X_circle, y_circle, 
                                    title="Linear Kernel SVM", ax=axes[0])

# RBF SVM
svm_rbf = KernelSVM(kernel='rbf', gamma=0.5, C=1.0)
svm_rbf.fit(X_circle, y_circle)

visualizer.plot_decision_boundary_2d(svm_rbf, X_circle, y_circle, 
                                    title="RBF Kernel SVM", ax=axes[1])

plt.tight_layout()
plt.show()

print(f"Linear SVM accuracy: {np.mean(svm_linear.predict(X_circle) == y_circle):.3f}")
print(f"RBF SVM accuracy: {np.mean(svm_rbf.predict(X_circle) == y_circle):.3f}")

## 4. Optimization Problem {#optimization}

### 4.1 Soft Margin SVM

For non-separable data, we introduce slack variables $\xi_i$:

$$\min_{w,b,\xi} \frac{1}{2}||w||^2 + C \sum_{i=1}^n \xi_i$$
$$\text{subject to: } y_i(w^T x_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0$$

The parameter $C$ controls the trade-off between margin maximization and classification error.

### 4.2 Sequential Minimal Optimization (SMO)

SMO is an efficient algorithm for solving the SVM optimization problem by:
1. Selecting two Lagrange multipliers to optimize
2. Solving the sub-problem analytically
3. Repeating until convergence

In [ ]:
# Demonstrate effect of C parameter
def demonstrate_c_parameter():
    # Create noisy data
    np.random.seed(42)
    X_noisy = np.vstack([
        np.random.multivariate_normal([2, 2], [[1, 0.5], [0.5, 1]], 50),
        np.random.multivariate_normal([6, 6], [[1, 0.5], [0.5, 1]], 50)
    ])
    y_noisy = np.hstack([np.ones(50), -np.ones(50)])
    
    # Add some noise/outliers
    outliers = np.array([[1, 6], [7, 2]])
    outlier_labels = np.array([1, -1])
    
    X_noisy = np.vstack([X_noisy, outliers])
    y_noisy = np.hstack([y_noisy, outlier_labels])
    
    C_values = [0.1, 1.0, 10.0]
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, C in enumerate(C_values):
        svm = LinearSVM(C=C, max_iter=1000)
        svm.fit(X_noisy, y_noisy)
        
        visualizer.plot_decision_boundary_2d(svm, X_noisy, y_noisy, 
                                           title=f"C = {C}", ax=axes[idx])
        
        # Highlight outliers
        axes[idx].scatter(outliers[:, 0], outliers[:, 1], 
                         c=['red', 'blue'], marker='x', s=200, linewidths=3,
                         label='Outliers')
        axes[idx].legend()
    
    plt.tight_layout()
    plt.show()

demonstrate_c_parameter()

## 5. Visual Intuition {#visual-intuition}

Let's visualize how SVMs work in different scenarios and understand the geometric intuition.

In [ ]:
# 3D visualization of kernel transformation
def visualize_kernel_transformation():
    # Create 2D data that's not linearly separable
    np.random.seed(42)
    
    # Create XOR-like data
    X_2d = np.array([
        [1, 1], [1, -1], [-1, 1], [-1, -1],  # Core points
        [0.8, 0.8], [0.8, -0.8], [-0.8, 0.8], [-0.8, -0.8],  # Close points
        [1.2, 1.2], [1.2, -1.2], [-1.2, 1.2], [-1.2, -1.2],  # Far points
    ])
    y_2d = np.array([1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1])
    
    # Transform to 3D using polynomial features (x1, x2, x1*x2)
    X_3d = np.column_stack([
        X_2d[:, 0],
        X_2d[:, 1], 
        X_2d[:, 0] * X_2d[:, 1]
    ])
    
    fig = plt.figure(figsize=(16, 6))
    
    # 2D plot
    ax1 = fig.add_subplot(121)
    colors = ['red' if label == 1 else 'blue' for label in y_2d]
    ax1.scatter(X_2d[:, 0], X_2d[:, 1], c=colors, s=100, alpha=0.7)
    ax1.set_xlabel('X1')
    ax1.set_ylabel('X2')
    ax1.set_title('Original 2D Space (Not Linearly Separable)')
    ax1.grid(True, alpha=0.3)
    
    # 3D plot
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], c=colors, s=100, alpha=0.7)
    ax2.set_xlabel('X1')
    ax2.set_ylabel('X2')
    ax2.set_zlabel('X1 * X2')
    ax2.set_title('Transformed 3D Space (Linearly Separable)')
    
    # Add separating plane in 3D
    xx, yy = np.meshgrid(np.linspace(-2, 2, 10), np.linspace(-2, 2, 10))
    zz = np.zeros_like(xx)  # Plane at z=0 separates the classes
    ax2.plot_surface(xx, yy, zz, alpha=0.3, color='green')
    
    plt.tight_layout()
    plt.show()

visualize_kernel_transformation()

## 6. Practical Considerations {#practical}

### 6.1 Hyperparameter Selection

Key hyperparameters to tune:
- **C**: Regularization parameter (higher C = less regularization)
- **γ (gamma)**: RBF kernel parameter (higher γ = more complex boundary)
- **Kernel choice**: Linear, polynomial, RBF, sigmoid

### 6.2 Preprocessing

- **Feature scaling**: SVMs are sensitive to feature scales
- **Feature selection**: Remove irrelevant features
- **Handle missing values**: Imputation strategies

### 6.3 Model Selection

Use cross-validation to select:
- Best kernel function
- Optimal hyperparameters
- Model complexity

In [ ]:
# Hyperparameter sensitivity analysis
def hyperparameter_sensitivity():
    from sklearn.model_selection import validation_curve
    from sklearn.svm import SVC
    from sklearn.datasets import make_classification
    
    # Generate dataset
    X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, 
                              n_informative=2, n_clusters_per_class=1, 
                              random_state=42)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # C parameter sensitivity
    C_range = np.logspace(-3, 3, 7)
    train_scores, test_scores = validation_curve(
        SVC(kernel='rbf', gamma='scale'), X, y, param_name='C', 
        param_range=C_range, cv=5, scoring='accuracy'
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    axes[0].semilogx(C_range, train_mean, 'o-', color='blue', label='Training')
    axes[0].fill_between(C_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    axes[0].semilogx(C_range, test_mean, 'o-', color='red', label='Validation')
    axes[0].fill_between(C_range, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')
    axes[0].set_xlabel('C Parameter')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('C Parameter Sensitivity')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Gamma parameter sensitivity
    gamma_range = np.logspace(-4, 1, 6)
    train_scores, test_scores = validation_curve(
        SVC(kernel='rbf', C=1.0), X, y, param_name='gamma', 
        param_range=gamma_range, cv=5, scoring='accuracy'
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    axes[1].semilogx(gamma_range, train_mean, 'o-', color='blue', label='Training')
    axes[1].fill_between(gamma_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    axes[1].semilogx(gamma_range, test_mean, 'o-', color='red', label='Validation')
    axes[1].fill_between(gamma_range, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')
    axes[1].set_xlabel('Gamma Parameter')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Gamma Parameter Sensitivity')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

hyperparameter_sensitivity()

## Summary

Key takeaways from SVM theory:

1. **Geometric Intuition**: SVMs find the hyperplane with maximum margin
2. **Mathematical Foundation**: Based on convex optimization theory
3. **Kernel Trick**: Enables non-linear classification without explicit feature mapping
4. **Support Vectors**: Only a subset of training points determine the model
5. **Regularization**: C parameter controls overfitting
6. **Preprocessing**: Feature scaling is crucial for SVM performance

### When to Use SVMs:
- ✅ High-dimensional data
- ✅ Clear margin of separation
- ✅ Small to medium datasets
- ✅ Non-linear relationships (with kernels)

### When NOT to Use SVMs:
- ❌ Very large datasets (slow training)
- ❌ Noisy data with overlapping classes
- ❌ Need probability estimates (requires calibration)
- ❌ Many irrelevant features